# 🚦 Analisis Data Eksploratif: Pola Lalu Lintas Kota Medan

 **Tujuan Notebook:**
# Notebook ini bertujuan untuk melakukan analisis eksploratif mendalam terhadap data trafik Kota Medan. Fokus utama adalah untuk memahami distribusi kecepatan kendaraan, mengidentifikasi pola temporal (harian, mingguan) dan spasial (ruas jalan), serta menemukan insight awal terkait potensi kemacetan.
#
# **Dataset yang Digunakan:** Data dari Silver Layer HDFS (gabungan trafik dan cuaca) atau Gold Layer (data agregat awal).
#
# **Tahapan Analisis:**
# 1. Setup Environment Spark & Pustaka
# 2. Pemuatan Data Trafik
# 3. Pembersihan Data Awal & Pemeriksaan Kualitas (spesifik untuk EDA)
# 4. Analisis Statistik Deskriptif Umum
# 5. Analisis Distribusi Kecepatan Kendaraan
# 6. Analisis Pola Temporal:
#    - Kecepatan rata-rata per jam dalam sehari
#    - Volume kendaraan per jam dalam sehari
#    - Perbandingan pola hari kerja vs. akhir pekan
# 7. Analisis Pola Spasial (Jika data memungkinkan):
#    - Kecepatan rata-rata per ruas jalan (`road_name`)
#    - Identifikasi ruas jalan dengan kecepatan rata-rata terendah (potensi macet)
# 8. Kesimpulan Awal dan Hipotesis


In [ ]:
# 1. Setup Environment Spark & Pustaka
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, hour, dayofweek, when, to_timestamp, date_format
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Inisialisasi SparkSession (sesuaikan dengan konfigurasi cluster Anda jika perlu)
spark = SparkSession.builder \
    .appName("EDATrafficPatternsMedan") \
    .config("spark.sql.repl.eagerEval.enabled", True) \ # Untuk output DataFrame Pandas-like di notebook
    .getOrCreate()

print("SparkSession berhasil dibuat!")

# ## 2. Pemuatan Data Trafik
# Memuat data dari HDFS Silver Layer (gabungan trafik dan cuaca) atau Gold Layer jika sudah ada agregasi awal.


In [ ]:
# Path ke data di HDFS (contoh dari Silver Layer)
silver_data_path = "/user/hive/warehouse/kemacetan_medan/silver/trafik_cuaca/data_gabungan.parquet"
# Atau dari Gold Layer
# gold_data_path = "/user/hive/warehouse/kemacetan_medan/gold/trafik_cuaca_agg/agregat_kemacetan.orc"

try:
    df_traffic = spark.read.parquet(silver_data_path)
    # Jika dari Gold Layer (ORC)
    # df_traffic = spark.read.orc(gold_data_path)
    print(f"Data berhasil dimuat. Jumlah baris: {df_traffic.count()}")
    df_traffic.printSchema()
    df_traffic.show(5, truncate=False)
except Exception as e:
    print(f"Error memuat data: {e}")
    # Hentikan eksekusi atau tangani error

# ## 3. Pembersihan Data Awal & Pemeriksaan Kualitas
# Meskipun data di Silver Layer sudah dibersihkan, kita mungkin perlu melakukan pemeriksaan atau pembersihan tambahan spesifik untuk EDA.


In [ ]:
# Cek nilai null pada kolom-kolom penting untuk analisis ini
print("Jumlah nilai null per kolom:")
df_traffic.select([count(when(col(c).isNull(), c)).alias(c) for c in ["timestamp", "speed_kmh", "road_name", "latitude", "longitude"]]).show()

# Filter kecepatan yang tidak realistis (misalnya, speed_kmh <= 0 atau sangat tinggi > 150 km/h)
df_traffic_filtered = df_traffic.filter((col("speed_kmh") > 0) & (col("speed_kmh") <= 150))
print(f"Jumlah baris setelah filter kecepatan: {df_traffic_filtered.count()}")

# Konversi timestamp jika belum atau ekstrak komponen waktu
# Diasumsikan 'timestamp' sudah bertipe Timestamp dari proses Bronze-to-Silver
df_traffic_filtered = df_traffic_filtered \
    .withColumn("jam", hour(col("timestamp"))) \
    .withColumn("hari_ke_minggu", dayofweek(col("timestamp"))) # 1=Minggu, 2=Senin, ..., 7=Sabtu


# ## 4. Analisis Statistik Deskriptif Umum
# Melihat statistik dasar dari kolom numerik utama.

In [ ]:
print("Statistik Deskriptif untuk Kecepatan (speed_kmh):")
df_traffic_filtered.select("speed_kmh").describe().show()
# Anda juga bisa melakukan ini untuk latitude, longitude jika relevan untuk analisis spasial awal


# ## 5. Analisis Distribusi Kecepatan Kendaraan
# Memahami sebaran kecepatan kendaraan secara keseluruhan.


In [ ]:
# Untuk visualisasi, lebih mudah mengambil sampel atau mengagregasi ke Pandas jika datanya besar
# Hati-hati jika data sangat besar, ini bisa menyebabkan OutOfMemoryError
speed_distribution_pd = df_traffic_filtered.select("speed_kmh").sample(False, 0.1).toPandas() # Ambil sampel 10%

plt.figure(figsize=(10, 6))
sns.histplot(speed_distribution_pd['speed_kmh'], bins=50, kde=True)
plt.title('Distribusi Kecepatan Kendaraan (speed_kmh) di Kota Medan')
plt.xlabel('Kecepatan (km/h)')
plt.ylabel('Frekuensi')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# **Interpretasi Distribusi Kecepatan:**
# * Dari grafik di atas, kita dapat mengamati bahwa sebagian besar kecepatan kendaraan berkisar antara X km/jam hingga Y km/jam.
# * Terdapat puncak distribusi di sekitar Z km/jam, yang mungkin merepresentasikan kecepatan jelajah normal di kondisi lalu lintas tertentu.
# * Adanya *tail* ke arah kecepatan rendah menunjukkan adanya kejadian perlambatan atau kemacetan.


# ## 6. Analisis Pola Temporal
# Menganalisis bagaimana kecepatan dan volume lalu lintas berubah berdasarkan waktu (jam dalam sehari, hari dalam seminggu).


# ### 6.1 Kecepatan Rata-rata per Jam dalam Sehari


In [ ]:
avg_speed_per_hour = df_traffic_filtered.groupBy("jam") \
    .agg(avg("speed_kmh").alias("avg_speed")) \
    .orderBy("jam") \
    .toPandas()

plt.figure(figsize=(12, 6))
sns.lineplot(data=avg_speed_per_hour, x="jam", y="avg_speed", marker='o')
plt.title('Kecepatan Rata-rata Kendaraan per Jam di Kota Medan')
plt.xlabel('Jam dalam Sehari (0-23)')
plt.ylabel('Kecepatan Rata-rata (km/h)')
plt.xticks(range(0, 24))
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# **Interpretasi Kecepatan Rata-rata per Jam:**
# * Grafik menunjukkan adanya penurunan kecepatan signifikan pada jam-jam sibuk pagi (sekitar pukul X-Y) dan sore (sekitar pukul A-B), yang mengindikasikan periode kemacetan.
# * Kecepatan cenderung lebih tinggi pada jam-jam malam hingga dini hari.


# ### 6.2 Volume Kendaraan (Jumlah Pengamatan) per Jam dalam Sehari

In [ ]:
volume_per_hour = df_traffic_filtered.groupBy("jam") \
    .agg(count("*").alias("vehicle_count")) \
    .orderBy("jam") \
    .toPandas()

plt.figure(figsize=(12, 6))
sns.barplot(data=volume_per_hour, x="jam", y="vehicle_count", color='skyblue')
plt.title('Volume Kendaraan (Jumlah Pengamatan GPS) per Jam di Kota Medan')
plt.xlabel('Jam dalam Sehari (0-23)')
plt.ylabel('Jumlah Pengamatan Kendaraan')
plt.xticks(range(0, 24))
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.show()

# **Interpretasi Volume Kendaraan per Jam:**
# * Pola volume kendaraan sejalan dengan pola kecepatan, di mana puncak volume terjadi pada jam-jam sibuk.
# * Volume terendah tercatat pada dini hari.

# ### 6.3 Perbandingan Pola Hari Kerja vs. Akhir Pekan

In [ ]:
df_traffic_filtered = df_traffic_filtered.withColumn("tipe_hari",
    when((col("hari_ke_minggu") >= 2) & (col("hari_ke_minggu") <= 6), "Hari Kerja") # Senin-Jumat
    .otherwise("Akhir Pekan") # Sabtu-Minggu
)

avg_speed_hourly_daytype = df_traffic_filtered.groupBy("jam", "tipe_hari") \
    .agg(avg("speed_kmh").alias("avg_speed")) \
    .orderBy("tipe_hari", "jam") \
    .toPandas()

plt.figure(figsize=(14, 7))
sns.lineplot(data=avg_speed_hourly_daytype, x="jam", y="avg_speed", hue="tipe_hari", marker='o')
plt.title('Perbandingan Kecepatan Rata-rata per Jam: Hari Kerja vs. Akhir Pekan')
plt.xlabel('Jam dalam Sehari (0-23)')
plt.ylabel('Kecepatan Rata-rata (km/h)')
plt.xticks(range(0, 24))
plt.legend(title='Tipe Hari')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# **Interpretasi Pola Hari Kerja vs. Akhir Pekan:**
# * Terlihat perbedaan signifikan pada pola kecepatan. Pada hari kerja, penurunan kecepatan pada jam sibuk lebih tajam.
# * Pada akhir pekan, kecepatan rata-rata cenderung lebih stabil atau penurunan pada jam sibuk tidak sedrastis hari kerja, namun mungkin ada puncak aktivitas di jam berbeda (misalnya, malam akhir pekan).


# ## 7. Analisis Pola Spasial (Contoh Sederhana)
# Jika data `road_name` cukup baik, kita bisa melihat kecepatan rata-rata per ruas jalan.


In [ ]:
# Hati-hati jika jumlah road_name sangat banyak, visualisasi bisa jadi tidak efektif
# Ambil top N ruas jalan dengan pengamatan terbanyak atau kecepatan terendah
top_n = 20
avg_speed_per_road = df_traffic_filtered.filter(col("road_name") != "Unknown") \
    .groupBy("road_name") \
    .agg(
        avg("speed_kmh").alias("avg_speed"),
        count("*").alias("observation_count")
    ) \
    .filter(col("observation_count") > 50) # Filter ruas jalan dengan observasi yang cukup
    .orderBy(col("avg_speed")) \
    .limit(top_n) \
    .toPandas()

if not avg_speed_per_road.empty:
    plt.figure(figsize=(12, 8))
    sns.barplot(data=avg_speed_per_road, y="road_name", x="avg_speed", palette="viridis")
    plt.title(f'Top {top_n} Ruas Jalan dengan Kecepatan Rata-rata Terendah (Potensi Macet)')
    plt.xlabel('Kecepatan Rata-rata (km/h)')
    plt.ylabel('Nama Ruas Jalan')
    plt.tight_layout() # Agar label tidak terpotong
    plt.show()
else:
    print("Tidak cukup data ruas jalan yang memenuhi kriteria untuk ditampilkan.")


# **Interpretasi Analisis Spasial:**
# * Dari grafik (jika ada), kita dapat mengidentifikasi beberapa ruas jalan spesifik yang memiliki kecepatan rata-rata paling rendah, yang mengindikasikan bahwa ruas-ruas jalan tersebut adalah titik rawan kemacetan.
# * Informasi ini sangat berharga untuk investigasi lebih lanjut atau sebagai fitur dalam model.


# ## 8. Kesimpulan Awal dan Hipotesis dari EDA Trafik
# Berdasarkan analisis eksploratif data trafik ini, beberapa kesimpulan awal dan hipotesis dapat ditarik:
# 1.  **Pola Temporal Jelas**: Terdapat pola harian yang jelas terkait kecepatan dan volume lalu lintas, dengan jam sibuk pagi dan sore menunjukkan penurunan kecepatan dan peningkatan volume. Perbedaan juga terlihat antara hari kerja dan akhir pekan.
# 2.  **Identifikasi Potensi Kemacetan**: Kecepatan kendaraan yang rendah pada jam-jam tertentu dan di ruas jalan tertentu mengindikasikan area dan waktu potensi kemacetan.
# 3.  **Kualitas Data**: Meskipun telah dibersihkan di Silver Layer, data kecepatan masih memerlukan filter untuk menghilangkan nilai yang tidak realistis agar analisis lebih akurat. Keberadaan `road_name` yang "Unknown" perlu diperhatikan.
# 4.  **Hipotesis untuk Pemodelan**:
#     * Fitur waktu (jam, hari dalam seminggu, tipe hari) akan menjadi prediktor penting.
#     * Nama ruas jalan (setelah di-encode) kemungkinan akan signifikan.
#     * Kecepatan rata-rata pada jam sebelumnya (fitur lag) bisa meningkatkan akurasi prediksi.
#
# Analisis lebih lanjut pada notebook `eda_weather_impact.ipynb` akan menggali bagaimana faktor cuaca berinteraksi dengan pola-pola ini. Insight dari `feature_insights.ipynb` akan digunakan untuk memvalidasi dan memilih fitur terbaik untuk model.


In [ ]:
# Hentikan SparkSession di akhir notebook
spark.stop()